In [7]:
import requests
import pandas as pd
import numpy as np 
from keys import * #Se importan los paquetes necesarios para poder leer y arrojar resultados en formatos optimos, en el caso de keys
                    #Se importa el archivo y se cambia su extensión a .py
city = "Montréal"
country = "CA"
response = requests.get(f'http://api.openweathermap.org/data/2.5/forecast/?q={city},{country}&appid={OWM_key}&units=metric&lang=en')
# Se ingresan los datos con los cuales se realizará la busqueda, ciudad, país y los datos que se extraerán de la página

In [6]:
data = response.json()
print (data)

# Extract forecast list
forecast_list = data.get('list', [])

# Prepare lists for DataFrame columns
times = []
temperatures = []
humidities = []
weather_statuses = []
wind_speeds = []
rain_volumes = []
snow_volumes = []

for entry in forecast_list:
    times.append(entry.get('dt_txt', np.nan))
    temperatures.append(entry.get('main', {}).get('temp', np.nan))
    humidities.append(entry.get('main', {}).get('humidity', np.nan))
    weather_statuses.append(entry.get('weather', [{}])[0].get('main', np.nan))
    wind_speeds.append(entry.get('wind', {}).get('speed', np.nan))
    rain_volumes.append(entry.get('rain', {}).get('3h', np.nan))
    snow_volumes.append(entry.get('snow', {}).get('3h', np.nan))

# Create DataFrame
df = pd.DataFrame({
    'time': times,
    'temperature': temperatures,
    'humidity': humidities,
    'weather_status': weather_statuses,
    'wind_speed': wind_speeds,
    'rain_volume_3h': rain_volumes,
    'snow_volume_3h': snow_volumes
})

print(df.head())

{'cod': '200', 'message': 0, 'cnt': 40, 'list': [{'dt': 1758758400, 'main': {'temp': 20.24, 'feels_like': 20.12, 'temp_min': 19.03, 'temp_max': 20.24, 'pressure': 1019, 'sea_level': 1019, 'grnd_level': 1015, 'humidity': 69, 'temp_kf': 1.21}, 'weather': [{'id': 803, 'main': 'Clouds', 'description': 'broken clouds', 'icon': '04n'}], 'clouds': {'all': 75}, 'wind': {'speed': 2.56, 'deg': 58, 'gust': 6.1}, 'visibility': 10000, 'pop': 0, 'sys': {'pod': 'n'}, 'dt_txt': '2025-09-25 00:00:00'}, {'dt': 1758769200, 'main': {'temp': 19.15, 'feels_like': 18.87, 'temp_min': 16.98, 'temp_max': 19.15, 'pressure': 1019, 'sea_level': 1019, 'grnd_level': 1015, 'humidity': 67, 'temp_kf': 2.17}, 'weather': [{'id': 803, 'main': 'Clouds', 'description': 'broken clouds', 'icon': '04n'}], 'clouds': {'all': 83}, 'wind': {'speed': 1.89, 'deg': 51, 'gust': 2.87}, 'visibility': 10000, 'pop': 0, 'sys': {'pod': 'n'}, 'dt_txt': '2025-09-25 03:00:00'}, {'dt': 1758780000, 'main': {'temp': 17.45, 'feels_like': 17.13, 't